### This file will use the Rotational Forest Classifier from the Aeon library

In [24]:
from _Data_Processing import *

'''
this function returns 7 objects:
	data: sets of data as X_train, X_val, X_ind, y_train, y_val, y_ind
	feature_list: list of all feature names with a column index (dict)
'''
X_train, X_val, X_ind,\
y_train, y_val, y_ind,\
feature_subsets =\
preprocess_data(
	file_name   =		'betaset_tmp.csv'
	,indp_size  =		0.05
	,test_size  =		0.15
	,shfl_splt	=		False
	,t_start    =		570
	,t_end		=		720
	,mod_type	=		'Classification'
	,target_t	=		15
	,num_class	=		2
	,split_val	=		5
	,verbose	=		1
	,scaler		=		'Standard'
	,frmt_lstm	=		False
	,time_steps =		5
	,keep_price =		False
)

Trying to load CSV file into DataFrame...Success.
Trying to drop unused targets...Success.
Trying to collect indices of wanted times...Success.
Trying to drop price features...Success...

# of Samples:	226576

# of Features:	436

Target:		tc_2c_15m

Trying to split DataFrame into X and y...Success.
Trying to collect all feature names and indices...Success.
Trying to clean up...Success.
Trying to encode y and make class weights...Failed [NON-FATAL: NOT IMPLEMENTED]
Trying to standardize all featurespace from training featurespace...Success.
Trying to drop unwanted time-range samples...Success.
	201633 Samples Dropped.

Trying to split X and y into Train/Validation/Independent...Success.
Trying to clean up...Success.
X_train:	(19954, 436).
y_train:	(19954,).
X_val:  	(3741, 436).
y_val:  	(3741,).
X_ind:  	(1248, 436).
y_ind:  	(1248,).
Terminating.


In [26]:
from _Data_Rotating import *

'''
This function returns 1 list of rotated partitions of the training set
'''

X_train_parts = rotate_partitions(
	X				=	X_train
	,n_feat_parts	=	1
	,feat_subsets	=	feature_subsets
	,feat_part_type	=	'by_subset'
	,fraction_feats	=	1.0
	,no_feat_overlap=	False
	,feats_for_all	=	[]
	,rotation_type	=	'None'
	,rotation_filter=	False
	,filter_type	=	'Retention'
	,filter_value	=	1.0
    ,n_sample_parts	=	1
    ,smpl_part_type	=	'Even'
    ,sample_shuffle	=	False
)

'''
all partitions
	X_train_parts
all sample partitions of THE featurespace partition i
	X_train_parts[i]
The sample partition j of THE featurespace partition i
	X_train_parts[i][j] -- -- -- This is now a set of trainable data
'''

'\nall partitions\n\tX_train_parts\nall sample partitions of THE featurespace partition i\n\tX_train_parts[i]\nThe sample partition j of THE featurespace partition i\n\tX_train_parts[i][j] -- -- -- This is now a set of trainable data\n'

In [4]:
from sklearn.tree import DecisionTreeClassifier
from aeon.classification.sklearn import RotationForestClassifier

def new_dt(max_depth):
    return DecisionTreeClassifier(max_depth=max_depth)

#56%accuracy with these stats
model = RotationForestClassifier(base_estimator=new_dt(max_depth=4),
                                    n_estimators=4
                                    ,min_group=1
                                    ,max_group=20
                                    ,remove_proportion= 0.3
                                    ,contract_max_n_estimators=500
                                    ,random_state=42,n_jobs=-1)



In [13]:
X_partitions=split_by_features(
					#using original X input !!!
					X=X_train
					,feat_sbst=feature_subsets
					,num_parts=1
					,part_type='by_subset'
					,full_excl=False
					,univ_incl=[]
					,part_sbst=1.0
					)

In [14]:
X_rotations = data_rotation(
	X_partitions=X_partitions
	,rotn_type="None"
	,filter_=False
)

In [15]:
X_dubpart = split_by_samples(
	X_partitions=X_rotations
	,num_parts=1
	,splt_type='Even'
	,shuffle=False
)

In [27]:
model.fit(X_train_parts[0][0], y_train)

RotationForestClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),
                         max_group=20, min_group=1, n_estimators=4, n_jobs=-1,
                         random_state=42, remove_proportion=0.3)

In [28]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_predt = model.predict(X_train)
y_predv = model.predict(X_val)
y_predi = model.predict(X_ind)
cmt = confusion_matrix(y_predt, y_train)
cmv = confusion_matrix(y_predv, y_val)
cmi = confusion_matrix(y_predi, y_ind)
#print(cm2)
print(accuracy_score(y_predt, y_train))
print(accuracy_score(y_predv, y_val))
print(accuracy_score(y_predi, y_ind))

0.6063947078280044
0.5490510558674151
0.5392628205128205


In [32]:
from _Prediction_Fusion import *

y_pred = pred_proba_fusion(
	models=models
	,type_model=type(models[0])
	,X_test=X_val
	,fusion_type='Popular'
)

numpy.ndarray

In [4]:
print(len(X_train[0]))
print(len(X_train_parts[0][0][0]))

436
436


In [13]:
var1 = ['123','234','345']
var2 = 'hello'
var3 = []
var4 = 4

for string in var1:
	print(string)

123
234
345
